# Will's cone detection notebook

## Purpose

I'm not very good at java tbh, so I'll do my code things here.

In [ ]:
from functools import reduce
from math import ceil
import cv2
import numpy as np
import matplotlib.pyplot as plt
from num2words import num2words

# grab relative path for loading images
import os
notebook_path = os.path.abspath("Notebook.ipynb")

def compose(*f_arg_pairs):
    def cr(*fs):
        pair = fs[0]
        if len(fs) == 1:
            return lambda x:pair[0](x,*pair[1])
        return lambda x:pair[0](cr(*fs[1:])(x),*pair[1])
    return cr(*reversed(f_arg_pairs))

def compose_gen(*f_arg_pairs):
    def cr(arg):
        pairs = [arg]
        for f_arg in f_arg_pairs:
            pairs.append(f_arg[0](pairs[-1], *f_arg[1]))
        return pairs
    return cr

digitalsum = compose(
    (int,()),
    (str,()),
    (list,()),
    (lambda x: (int(n) for n in x),()),
    (sum, ()),
)

# print(digitalsum(314159265358979323))


def add_image(*dispImage):
    wrap = 4
    if len(dispImage) > wrap: #wrap is the maximum amount of pictures before they get too small to see.
        x,y = wrap,ceil(len(dispImage)/wrap)
    else:
        x,y = len(dispImage),1
    fig = plt.figure(figsize=(6*x,8*y))
    for index, img in enumerate(dispImage):
        fig.add_subplot(y, x, index+1)
        # showing image
        plt.imshow(img)
        plt.axis('off')
        plt.title(num2words(index+1,True,to="ordinal").capitalize())

## Image processing functions

In [ ]:
def flood_from_pix(floodImg: np.ndarray, pixel: tuple = ()):
    h, w = floodImg.shape[:2]
    if not pixel: pixel = (w//2, h//2)
    mask = np.zeros((h+2,w+2), np.uint8)
    flat = floodImg.astype("uint8")
    cv2.floodFill(flat, mask, pixel, 255)
    return cv2.threshold(flat, 254, 255, cv2.THRESH_BINARY)[1]

def resize_image(dimImage: np.ndarray, newsize: tuple = ()):
    print(dims:=dimImage.shape)
    if newsize:
        rowedge = (dims[0]-newsize[0])//2
        coledge = (dims[1]-newsize[1])//2
        return dimImage[rowedge:dims[0]-rowedge,coledge:dims[1]-coledge]

    return dimImage

def tri_stack(stackImg: np.ndarray):
    return np.dstack([stackImg,stackImg,stackImg])

def extract_channel(extrImg: np.ndarray, channel: int = 0):
    return tri_stack(extrImg[:,:,channel])

def point(pointImg: np.ndarray, xy: tuple, clr: tuple):
    return cv2.circle(pointImg, xy, radius=5, color=clr, thickness=-1)

## Shape Pipeline

This pipeline takes a cone with a shape on it, and returns a mask over the shape.

In [ ]:
from pickle import dump

color_pipeline = lambda img, seed: compose_gen(
        (extract_channel, (0,)),
        # 200 is min, 250 is max. If theres too much bg noise, turn up 200. If the shape gets cut, turn down 250.
        (cv2.Canny, (400, 500)),
        (cv2.blur, ((5, 5),)),
        (flood_from_pix, (seed,)),
        # (cv2.Canny, (200, 250)),
        # (lambda x: cv2.threshold(x,254,255,cv2.THRESH_BINARY)[1],()),
        # (cv2.blur, ((5, 5),)),
        # (tri_stack, ()),
        # (point, (seed, (255,0,0))),
        (np.ndarray.astype,("uint8",)),
    )(img)

image = cv2.imread("Images/Shapes/Downscaled/trianglepb_downscaled_cone.jpg")
# webcam = cv2.VideoCapture(0)
# image = webcam.read()[1]
# webcam.release()
width, height, depth = image.shape

prep_pipeline = compose(
    (cv2.flip, (1,)),
    (cv2.cvtColor, (cv2.COLOR_RGB2BGR,)),
    # (resize_image, ((480,360),)),
)

image = prep_pipeline(image)

cnvimage = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
# cnvimages = color_pipeline(image,(250,245))
cnvimages = color_pipeline(image,(251,253))
# cnvimage = cv2.cvtColor(cnvimage, cv2.COLOR_HSV2RGB)
# add_image(cnvimages[5])
add_image(*cnvimages)
# cnvimage = cv2.cvtColor(cnvimage, cv2.COLOR_RGB2GRAY)

# image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
# cv2.imwrite("Images/Shapes V2/Normal/Pentagon.png",image)
# cv2.imwrite("Images/Shapes V2/Processed/Pentagon.png",cnvimage)

dump(image, open("temp/wimage",'wb+'))
dump(cnvimages[-1], open("temp/cnvimage",'wb+'))

### Grab edges

Take the edges, find the contours, and then approximate as a polygon.

In [ ]:
from pickle import load

# image = load(open("temp/image",'rb'))
image = load(open("temp/wimage",'rb'))
cnvimage = load(open("temp/cnvimage",'rb'))
# cvnimage = cnvimages[-1]

    # contours, _ = cv2.findContours(made_im, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # return cv2.approxPolyDP(contours[contour],20,True),made_im

# poly, outimage = count_poly_seed((height//2,width//2))

polys = []
for contour in cv2.findContours(cnvimage, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]:
    # contour = contour[0]
    if len(contour) <= 10: continue
    if len(poly:=cv2.approxPolyDP(contour,20,True)) > 1:
        polys.append(poly)

# poly, outimage = count_poly_seed((height//2,width//2))

# approxpolydp PER contour, filter out 1's
cnv_draws = []
for poly in polys:
    cnv_draw = np.zeros(cnvimage.shape,dtype=np.uint8)
    cnv_draw = tri_stack(cnv_draw)
    cnv_draw = cv2.polylines(cnv_draw,[poly],True,(0,0,255),3)
    # cnv_draw = cv2.drawContours(cnv_draw,poly,-1,(0,0,255),3)
    cnv_draws.append(cnv_draw)

cnvimage = tri_stack(cnvimage)
cnvimage = cv2.drawContours(cnvimage,polys,-1,(0,255,0),5)

# add_image(cnv_draws[1])
add_image(image, cnvimage, *cnv_draws)

## Get middle pixel
This will get the middle pixel location, from there we can do an additional color analysis

In [ ]:
poly = polys[0]

def average_pix(polycontour: np.ndarray):
    return tuple(sum(polycontour[:,:,x])[0]//len(polycontour) for x in (0,1))

print(*average_pix(poly))

## Putting it all together

I will combine all of the previous cells into one big pipeline that should, in theory, detect shapes on cones in a robust way.

In [ ]:
from math import sqrt

def get_shape(input_image: np.ndarray):
    '''Takes an input image, and tries to \
       return the number of sides in a \
       detected polygon. If this fails, \
       `return None`. `input_image` is in \
       HSV.'''
    
    # Variables used throughout the pipeline.
    im_h, im_w, _ = input_image.shape

    fill_pipe = lambda img, seed: compose_gen(
        (extract_channel, (2,)),
        # 400 is min, 500 is max. If theres too much bg noise, turn up 400. If the shape gets cut, turn down 500.
        (cv2.Canny, (200, 200)),
        (cv2.blur, ((5, 5),)),
        (flood_from_pix, (seed,)),
        (np.ndarray.astype,("uint8",)),
    )(img)

    pipe_b = lambda img, shape: compose_gen(
        (point, (new_seed,(255,0,0))),
        (cv2.polylines, [polys[0]],True,(0,0,255,3)),
    )

    full_pipe = fill_pipe(input_image,(im_w//2,im_h//2))\

    cnvimage = full_pipe[-1]

    def get_polys(poly_image: np.ndarray):
        polys = []
        for contour in cv2.findContours(poly_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]:
            # if len(contour) <= 10: continue
            if len(poly:=cv2.approxPolyDP(contour,20,True)) > 1:
                polys.append(poly)
        return polys
    
    polys = get_polys(cnvimage)
    
    # distance formula
    def dst_from_center(x,y):
        return sqrt((x1:=x-im_w//2)*x1 + (y1:=y-im_h//2)*y1)

    # Per shape, get average point distance to the center
    avg_dists = [sum(dst_from_center(x,y) for x,y in poly[:,0])/len(poly) for poly in polys]

    # with the index of the minimum shape, get the average pixel
    new_seed = average_pix(polys[avg_dists.index(min(avg_dists))])

    print(new_seed)
    
    cnvimage = tri_stack(cnvimage)
    cnvimages = fill_pipe(cnvimage,new_seed)

    polys = get_polys(cnvimages[-1])
    # Image is ready
    return len(polys[0]),full_pipe,cnvimages,new_seed

# image = cv2.imread("Images/Shapes/Downscaled/squarebb_downscaled_cone.jpg")
image = load(open("temp/wimage",'rb'))
# webcam = cv2.VideoCapture(0)
# image = webcam.read()[1]
# webcam.release()

# dump(image, open("temp/wimage",'wb+'))


image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

prep_pipeline = compose(
    (resize_image, ((480,360),)),
    (cv2.flip, (1,)),
    (cv2.cvtColor, (cv2.COLOR_RGB2HSV,)),
)

# add_image(image)
# point(cv2.flip(image,1),d,clr=(255,0,0))
a, b, c, d = get_shape(prep_pipeline(image))
print(a,image.shape)
add_image(image)
add_image(*b,*c)

## QR codes

I may be a fool, if this works I will cry.

In [ ]:
qrScanner = cv2.QRCodeDetector()
image = cv2.imread("Images/QR/coneqr1.jpg")
print(qrScanner.detectAndDecode(image))